<div style="color:red;background-color:black">
Diamond Light Source
<br style="color:red;background-color:antiquewhite"><h1>C Programming: gmake</h1><br>
©2000-21 Chris Seddon 
</div>

Consider the following set of C files:
<pre>
f1.c
f2.c
f3.c
f_main.c
</pre>

Let's take a look at the source:

In [ ]:
cd Makefiles
find -name '*.c' -exec cat {} \;

To compile the above files and produce a set of object files we need to:

In [ ]:
gcc -c f1.c
gcc -c f2.c
gcc -c f3.c
gcc -c f_main.c
ls -l *.o

To complete the link and produce an executable:

In [ ]:
gcc f1.o f2.o f3.o f_main.o -o f_main.exe

Let's check it worked:

In [ ]:
f_main.exe

The `gmake` utility defines a file called `Makefile` which is a recipe for creating the `main.exe` executable.  A makefile consists of a set of targets and rules:

In the above, each file we created is called a <b>target</b>.  The following targets were created:
<pre>
f1.o
f2.o
f3.o
f_main.o
main.exe
</pre>

Each of the above commands is called a <b>rule</b> in `gmake`.  The following rules were defined:
<pre>
gcc -c f1.c
gcc -c f2.c
gcc -c f3.c
gcc -c f_main.c
gcc f1.o f2.o f3.o f_main.o -o main.exe
</pre>

Lets look at the Makefile.  I've pre-prepared a several interim makefiles for this tutorial:
<pre>
Makefile.&lt;n&gt;
</pre>
so as I can easily change our makefile.

In [ ]:
cp Makefile.1 Makefile
cat Makefile

The makefile consists of a set of targets and the rules to create each target.  The rules are in reverse temporal order.  

After each target, there is a list of its dependencies.  For example:
<pre>
f1.o: f1.c f.h
</pre>
which states that `f1.o` is dependent on `f1.c` and `f1.h`.  This means if we change `f1.c` or `f.h` we need to rebuild `f1.o`.

We can use the makefile to rebuild the `f_main.exe` target by issuing the command:

In [ ]:
make

`gmake` has nothing to do because the target is already built.  

By default, `gmake` tries to build the first target defined in the makefile (`f_main.exe`).  To get `gmake` to build this target we need to first delete all the targets and then rerun `make`.

In [ ]:
rm *.o *.exe
make
ls -l f_main.exe

Actually we can define a rule to remove the targets (usually called `clean`).  Its convenient to supress error messages on this target.

The new makefile now reads:

In [ ]:
cp Makefile.2 Makefile
cat Makefile

The hyphen in front of `rm` stops `gmake` complaining in case of error and the `@` stops `gmake` from echoing the rule. 

To invoke the new target (`clean`), as its not the default (first) target, we need to specify it explicitly: 

In [ ]:
make clean
ls -l f*

`gmake` can be invoked with the `-n` option.  This is useful because it does a dry run.  It doesn't execute the rules, it merely prints the rules that will be invoked.  The other useful command I want to mention is `touch`.  `touch` updates the timestamp on a file.  This allows us to simulate changing a file without actually needing to edit the file.  `gmake` uses these timestamps to decide whether to rebuld a target.  

Consider:
<pre>
f1.o: f1.c f.h
</pre>
`gmake` will compare the timestamps of these 3 files.  If the target `f1.o` is not the oldest of the 3, then one of the dependencies has been edited and the target needs to be rebuilt.  

Let's try a few experiments;  try `touch`ing different files and then running `gmake`.
We run make silently to start each experiment:

`f1.o` depends on `f1.c`, so let's try touching `f1.c`

In [ ]:
make --silent
touch f1.c
make -n

So this means `f1.o` needs to rebuilt.  However, this will now make `f_main.exe` out of date, so this gets rebuilt as well.  Its usual for a rebuild of one target to trigger a rebuild of other targets.

Let's try something else:

In [ ]:
make --silent
touch f.h
make -n

Everything depends directly or indirectly on `f.h`, so touching this file rebuilds everything.

In [ ]:
make --silent
touch f2.o
make -n

Only the `f_main.exe` target depends on `f2.o`, so only this target gets rebuilt.

We can build individual targets by explictly naming them:

In [ ]:
make clean
make f1.o

Now we can build `f2.o`:

In [ ]:
make f2.o

So that's a basic makefile.  However, there is a lot of repetition in our makefile, so `gmake` defines a number of features to make things more concise.  

Let's look at an enhanced version of the makefile: 

In [ ]:
cp Makefile.3 Makefile
cat Makefile

We can use % as a wildcard to avoid repeating filenames.  The line:
<pre>
%.o: %c
</pre>
can be used instead of
<pre>
f1.o: f1.c
f2.o: f2.c
f3.o: f3.c
f_main.o: f_main.c
</pre>
Note that the macro `$<` represents the dependent file represented by `%.c`.  Hence for the case of f3 we have:
<pre>
%.c = f3.c
%.o = f3.o
$< = f3.c
</pre>


In [ ]:
make clean
make

There is a lot more to `gmake` than the above, but you should now have a basic understanding.  

Take a look at the <a href="https://www.gnu.org/software/make/manual/">`gmake` manual online</a>.